#### Дан набор рецензий к фильмам, задача автоматически определить какие из рецензий позитивные, а какие негативные. В качестве входных данных дано два файла train.csv и test.csv. Для тренировочного набора (train.csv) даны тексты рецензий, известна разметка и ID фильма к которому относится рецензия. Для тестового набора (test.csv) вам известен только текст рецензии. И вам требуется определить позитивная она или негативная.

#### В файле sample_submission.csv приведен пример файла для отправки на сервер. Он содержит два поля review_id и positive. Необходимо вывести бинарное значение (0 или 1) в поле positive в зависимости от того позитивная рецензия (1) или негативная (0). Важно: в этой задаче мы выводим не вероятность, а уже готовый ответ, то есть принимаются только значения 0 или 1.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

from sklearn.naive_bayes import MultinomialNB

## 1.загружаем датасеты

In [ ]:
train = pd.read_csv('train.csv',encoding='utf_8')
test = pd.read_csv('test.csv',encoding='utf_8')

train['text'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)
test['text'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)

train

,review_id,movie_id,text,label
0,0,0,Старая поговорка гласит: «Лучшие рассказы — э...,Good
1,1,0,"Самое сильное кино начала этого года, или кон...",Good
2,2,0,"Душевно. Когда противоположности встречаются,...",Good
3,3,0,"Об этом фильме я вообще ничего не знал, но не...",Good
4,4,0,"Как правило, история людей, которые прикованн...",Good
...,...,...,...,...
17110,16779,189,"Если вам интересно, что получится, если взять...",Bad
17111,16780,189,"Бывает так, собрался в кино, но на заранее вы...",Bad
17112,16781,189,"Некоторым понравился этот фильм, некоторым не...",Bad
17113,16782,189,"Когда я посмотрел некоторые отзывы, то мне ст...",Bad


## 2.далее
- определяем целевую переменную и входные параметры
- делим обучающую выборку train на тестовую и обучающую

In [ ]:
y = train['label'] #целевая переменная по оценке позитивности
x = train['text'] #входные параметры - текст

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train

50        12 разгневанных мужчин. Черно-белый фильм, в ...
13646     «Вот повезло же Ди Каприо — сыграть с самим К...
2606      Любовь бывает разная, но она случается с кажд...
9026      Среди фокусников, иллюзионистов всегда летали...
15599     Кто такой Уве Болл в наше время знают многие....
                               ...                        
9225      В далеком 1995 году в прокат вышел мультик от...
13123     Никто не смеет называть меня трусом, никто! ©...
9845      Этот фильм стоит на одном из самых почетных м...
10799     … мертв. Кризис среднего возраста заставляет ...
2732      Есть фильмы после просмотра которых остается ...
Name: text, Length: 13692, dtype: object

данные в таком виде нельзя передавать модели. Их надо привести к численному виду.

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y = le.fit_transform(y)
y

array([1, 1, 1, ..., 0, 0, 0])

Для преобразования текстовых сообщений воспользуемся `CountVectorizer`, работающему по принципу мешка слов (*bag of words*). Он имеет следующие гиперпараметры (т.е. те, которые задаются пользователем):

* `max_df` &mdash; максимальная доля сообщений, в которых может встречатся слово из словаря;
* `min_df` &mdash; минимальная доля сообщений, в которых может встречатся слово из словаря;
* `max_features` &mdash; максимальное возможное количество выбранных слов, они выбираются среди наиболее частых;
* `stop_words` &mdash;слова, которые не будут добавлены в словарь.

Построим векторное представление для наших сообщений. Для этого нужно объявить объект класса `CountVectorizer`. Далее применяем к обучающим данным функцию `fit_transform`, которая последовательно выполняет следующие функции:
* `fit` &mdash; обучение модели, в данном случае подсчет частот слов и определение словаря;
* `transform` &mdash; по существующему словарю преобразует сообщения в векторы.

In [ ]:
# Преобразование текста в набор численных вектором
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0.01, max_df=1.0)
x_vect_train = vectorizer.fit_transform(x_train)
x_vect_test = vectorizer.transform(x_test)

len(x_train), x_vect_train.shape

(13692, (13692, 2706))

Напечатаем весь мешок слов и их количество. Для наглядности, создадим таблицу, у которой названия колонок - это слова из мешка слов, а по сторкам стоят текстовые сообщения, слова в которых заменены на числа, которые представляют собой частоту встречи слова в данном сообщении. Нули на месте тех слов, которых нет в сообщении.

In [ ]:
pd.DataFrame(x_vect_train.A, columns=vectorizer.get_feature_names_out())

,10,100,11,12,15,20,2010,250,30,3d,...,явно,язык,якобы,яркая,яркие,яркий,ярким,ярких,ярко,ясно
0,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13687,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13688,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
13689,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 3.далее
- обучаем модель полиномиального наивного байесовского классификатора - подходит для классификации с дискретными функциями
### (хорошо подходит для классификации текстов).

In [ ]:
# Полиномиальный наивный байесовский классификатор подходит для классификации с дискретными функциями
# (хорошо подходит для классификации текстов).
model_MNB = MultinomialNB()
model_MNB.fit(x_vect_train, y_train)

MultinomialNB()

## 4. далее
- делаем предсказания

In [ ]:
y_pred = model_MNB.predict(x_vect_test)
y_pred

array(['Bad', 'Good', 'Good', ..., 'Good', 'Good', 'Good'], dtype='<U4')

Задача обучения состоит в том, чтобы подобрать параметры модели таким образом, чтобы она лучше всего описывала обучающие данные. Посмотрим, какие данные определены неверно и сколько их из всего количества точек датасета.

In [ ]:
check = pd.DataFrame({
    "y_test": y_test,
    "y_pred": y_pred,
})
check[check['y_test']!=check['y_pred']]

,y_test,y_pred
5149,Good,Bad
1483,Good,Bad
6528,Bad,Good
15781,Good,Bad
15210,Good,Bad
...,...,...
9940,Good,Bad
9679,Good,Bad
7229,Good,Bad
1875,Good,Bad


Оценим качество модели. Используем метрику "точность" (accuracy), которая определяется как доля верно классифицированных объектов.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9097283085013146

значение точности высокое - смело используем модель на тестовых данных

## 5. далее
- применение модели к тестовой выборке test

In [ ]:
test

,review_id,text
0,0,"Если честно, досмотреть фильм до конца было н..."
1,1,«Новые приключения Алёнушки и Ерёмы» December...
2,2,"Метель за окном, хочется чего то такого очень..."
3,3,Уже множество лестных слов было сказано в адр...
4,4,"Прекрасный, печальный, добрый и в то же время..."
...,...,...
15226,15226,Соглашусь со всеми рецензорами только в одном...
15227,15227,Вот решился на днях посмотреть эти самые «Рим...
15228,15228,Пираньи 3D вышли в 2010 году. Фильм хоть и не...
15229,15229,Уже после 10 минут просмотра этого фильма у м...


- сначала поработаем с данными - векторизируем текст

In [ ]:
x_TEST = test['text']
x_vect_TEST = vectorizer.transform(x_TEST)

len(x_TEST), x_vect_TEST.shape

(15231, (15231, 2706))

- делаем предсказания с помощью ранее обученной модели

In [ ]:
pred_res = model_MNB.predict(x_vect_TEST)
pred_res

array(['Bad', 'Bad', 'Good', ..., 'Bad', 'Bad', 'Good'], dtype='<U4')

- переведем стринговые данные в числовые через preprocessing

In [ ]:
le_res = preprocessing.LabelEncoder()

pred_res = le_res.fit_transform(pred_res)
pred_res

array([0, 0, 1, ..., 0, 0, 1])

- вставим полученные данные в тестовый датасет

In [ ]:
test.insert(2, column='label', value=pred_res)
test

,review_id,text,label
0,0,"Если честно, досмотреть фильм до конца было н...",0
1,1,«Новые приключения Алёнушки и Ерёмы» December...,0
2,2,"Метель за окном, хочется чего то такого очень...",1
3,3,Уже множество лестных слов было сказано в адр...,1
4,4,"Прекрасный, печальный, добрый и в то же время...",1
...,...,...,...
15226,15226,Соглашусь со всеми рецензорами только в одном...,1
15227,15227,Вот решился на днях посмотреть эти самые «Рим...,0
15228,15228,Пираньи 3D вышли в 2010 году. Фильм хоть и не...,0
15229,15229,Уже после 10 минут просмотра этого фильма у м...,0


## 6. далее
- разбираемся с формированием файла для сдачи

 - просмотрим, что в sample_submission

In [ ]:
sample_sub = pd.read_csv('sample_submission.csv')
sample_sub

,review_id,positive
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
15226,15226,0
15227,15227,0
15228,15228,0
15229,15229,0


чтобы вставить данные label из датафейма test в sample_submissions, нужно убедиться, что мы работаем с одинаковыми review_id (ну вдруг в какой-то таблице айдишники перемешаны или необходимого вообще нет или есть лишние айдишники). для этого удобнее использовать методы СУБД - join, но в питоне в pandas есть аналоги соединений. 
- с помощью merge соединим таблицы с левой стороны по одинаковым колонкам (review_id). 
- потом сразу же удалим лишнюю колонку positive, потому что там нет полезных данных.
- в конце переименуем колонки как в файле sample_submission.csv
- готово

In [ ]:
test_copy = test.copy().drop(['text'],axis=1) #сделаем копию датафрейма test, чтобы не утерять оригинальные данные
result = pd.merge(sample_sub, test_copy).drop('positive',axis=1)
result.columns = ['review_id', 'positive']
result

,review_id,positive
0,0,0
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
15226,15226,1
15227,15227,0
15228,15228,0
15229,15229,0


# сохраним ответ в файл

In [ ]:
result.to_csv('submission.csv', index=False)